In [43]:
%matplotlib inline

import os
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import pandas as pd
import numpy as np

import statsmodels.api as sm1 #Usa OLS com X como matriz de planejamento
import statsmodels.formula.api as sm  #Usa OLS como no R

print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
/Users/CaueCitrini/Documents/Graduação Insper/Segundo Semetre 2016:2/Ciência dos Dados/Projetos/Projeto 3


In [44]:
dados1 = pd.read_excel('indicator gapminder gdp_per_capita_ppp.xlsx',index_col=0)

In [45]:
dados2 = pd.read_excel('indicator private share of total health spending.xls.xlsx',index_col=0)

In [46]:
dados3 = pd.read_excel('Indicator_Improved sanitation total percent.xlsx',index_col=0)

In [47]:
dados4 = pd.read_excel('indicatorpolityiv.xlsx',index_col=0)

In [48]:
gdp_per_capita = pd.DataFrame()
gdp_per_capita["2012"] = dados1[2012]

private_share = pd.DataFrame()
private_share["2010"] = dados1[2010]

sanitation = pd.DataFrame()
sanitation["2010"] = dados1[2010]

democracy = pd.DataFrame()
democracy["2011"] = dados1[2011]

In [49]:
gdp_per_capita

,2012
GDP per capita,
Abkhazia,NaN
Afghanistan,1893
Akrotiri and Dhekelia,NaN
Albania,9811
Algeria,12779
American Samoa,NaN
Andorra,41926
Angola,7230
Anguilla,NaN


In [50]:
private_share

,2010
GDP per capita,
Abkhazia,NaN
Afghanistan,1637
Akrotiri and Dhekelia,NaN
Albania,9374
Algeria,12494
American Samoa,NaN
Andorra,38982
Angola,7047
Anguilla,NaN


In [51]:
sanitation

,2010
GDP per capita,
Abkhazia,NaN
Afghanistan,1637
Akrotiri and Dhekelia,NaN
Albania,9374
Algeria,12494
American Samoa,NaN
Andorra,38982
Angola,7047
Anguilla,NaN


In [52]:
democracy

,2011
GDP per capita,
Abkhazia,NaN
Afghanistan,1695
Akrotiri and Dhekelia,NaN
Albania,9640
Algeria,12606
American Samoa,NaN
Andorra,41958
Angola,7094
Anguilla,NaN
